In [4]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

In [10]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

In [12]:
train_numer_df = load_csv_df("train_numeric.csv")

In [14]:
test_numer_df = load_csv_df("test_numeric.csv")

In [26]:
def get_corr_matrix(df):
    return df.stat.corr(df, method="pearson")

In [34]:
#corr = get_corr_matrix(train_numer_df)
import pyspark.mllib.stat
#stat.Statistics.corr(train_numer_df)